##### 版權所有 2024 Google LLC.


In [ ]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Gemini API：Python 介面函式呼叫


<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/doggy8088/gemini-api-cookbook/blob/zh-tw/quickstarts/Function_calling.zh.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />在 Google Colab 中執行</a>
  </td>
</table>


函式呼叫可讓開發人員建立其程式碼中函式的說明，然後在請求中將說明傳遞給語言模型。模型的回應包括與說明匹配的函式名稱，以及呼叫它的參數。函式呼叫可讓你在生成式 AI 應用程式中將函式用作工具，你可以在單一請求中定義多個函式。

這本筆記本提供程式碼範例，協助你開始。


In [ ]:
!pip install -U -q google-generativeai  # Install the Python SDK

In [ ]:
import google.generativeai as genai

## 設定你的 API 權杖

要執行以下Cell，你的 API 權杖必須儲存在名為 `GOOGLE_API_KEY` 的 Colab 密碼中。如果你還沒有 API 權杖，或者你不確定如何建立 Colab 密碼，請參閱 [驗證](https://github.com/google-gemini/cookbook/blob/main/quickstarts/Authentication.ipynb) 快速入門作為範例。


In [ ]:
from google.colab import userdata

GOOGLE_API_KEY = userdata.get("GOOGLE_API_KEY")
genai.configure(api_key=GOOGLE_API_KEY)

## 函式呼叫基礎


若要運用函式呼叫，請在建立 [`GenerativeModel`](https://ai.google.dev/api/python/google/generativeai/GenerativeModel) 時，將一組函式傳遞至 `tools` 參數。該模型會使用函式名稱、文件字串、參數和參數類型註解，來判斷是否需要該函式以最佳方式回答提示。

> 重要：SDK 會將函式參數類型註解轉換為 API 可理解的格式 (`glm.FunctionDeclaration`)。API 僅支援有限的參數類型，而 Python SDK 的自動轉換僅支援其中一小部分： `AllowedTypes = int | float | bool | str | list['AllowedTypes'] | dict`


In [ ]:
def add(a: float, b: float):
    """returns a + b."""
    return a + b


def subtract(a: float, b: float):
    """returns a - b."""
    return a - b


def multiply(a: float, b: float):
    """returns a * b."""
    return a * b


def divide(a: float, b: float):
    """returns a / b."""
    return a / b


model = genai.GenerativeModel(
    model_name="gemini-1.0-pro", tools=[add, subtract, multiply, divide]
)

model

genai.GenerativeModel(
    model_name='models/gemini-1.0-pro',
    generation_config={},
    safety_settings={},
    tools=<google.generativeai.types.content_types.FunctionLibrary object at 0x7a99071b61d0>,
)

## 自動函式呼叫


函式呼叫可以自然地套用於 [多輪對話](https://ai.google.dev/api/python/google/generativeai/GenerativeModel#multi-turn)中，因為它們能捕捉使用者和模型之間的往返互動。Python SDK 的 [`ChatSession`](https://ai.google.dev/api/python/google/generativeai/ChatSession) 是一個聊天用的絕佳介面，因為它會為你處理對話記錄，而使用參數 `enable_automatic_function_calling` 可進一步簡化函式呼叫：


In [ ]:
chat = model.start_chat(enable_automatic_function_calling=True)

在啟用自動功能呼叫後，如果模型要求，`ChatSession.send_message` 會自動呼叫你的功能。

在以下範例中，結果看來只是一個包含正確答案的文字回應：


In [ ]:
response = chat.send_message(
    "I have 57 cats, each owns 44 mittens, how many mittens is that in total?"
)
response.text

'There are 2508 mittens in total.'

In [ ]:
57 * 44

2508

然而，透過查看聊天記錄，你可以看到對話的流程以及函式呼叫如何整合在其中。

`ChatSession.history` 屬性會儲存使用者與 Gemini 模型之間對話的順序紀錄。對話中的每個回合均會表示為 [`glm.Content`](https://ai.google.dev/api/python/google/ai/generativelanguage/Content) 物件，其中包含下列資訊：

*   **角色：** 識別內容是來自「使用者」還是「模型」。
*   **部分：** [`glm.Part`](https://ai.google.dev/api/python/google/ai/generativelanguage/Part) 物件清單表示訊息的個別組成部分。對於純文字模型，這些部分可以是：
    *   **文字：** 一般文字訊息。
    *   **函式呼叫** ([`glm.FunctionCall`](https://ai.google.dev/api/python/google/ai/generativelanguage/FunctionCall)): 由模型提出的請求，用於執行具有提供引數的特定函式。
    *   **函式回應** ([`glm.FunctionResponse`](https://ai.google.dev/api/python/google/ai/generativelanguage/FunctionResponse)): 由使用者在執行請求的函式後傳回的結果。

在先前的有手套計算範例中，歷程記錄會顯示下列順序：

1.  **使用者：** 詢問有關手套總數的問題。
1.  **模型：** 判定該乘法函式有幫助，並將 FunctionCall 請求傳送給使用者。
1.  **使用者：** `ChatSession` 會自動執行該函式 (因為已設定 `enable_automatic_function_calling`)，然後傳送一個具有計算結果的 `FunctionResponse` 回來。
1.  **模型：** 使用函式的輸出，形成最終答案，並以文字回應的方式呈現該結果。


In [ ]:
for content in chat.history:
    print(content.role, "->", [type(part).to_dict(part) for part in content.parts])
    print("-" * 80)

user -> [{'text': 'I have 57 cats, each owns 44 mittens, how many mittens is that in total?'}]
--------------------------------------------------------------------------------
model -> [{'function_call': {'name': 'multiply', 'args': {'a': 57.0, 'b': 44.0}}}]
--------------------------------------------------------------------------------
user -> [{'function_response': {'name': 'multiply', 'response': {'result': 2508.0}}}]
--------------------------------------------------------------------------------
model -> [{'text': 'There are 2508 mittens in total.'}]
--------------------------------------------------------------------------------


一般來說狀態圖是：

<img src="https://ai.google.dev/images/tutorials/function_call_state_diagram.png" alt="模型始終可以用文字或 FunctionCall 回覆。如果模型送出 FunctionCall，使用者必須以 FunctionResponse 回覆" width=50%>

模型可以在回傳文字回應前回覆多個 function calls，而 function calls 出現在文字回應之前。


## 手動 function 呼叫


若要進行更嚴格的控制，你可以自行處理來自模型的 [`glm.FunctionCall`](https://ai.google.dev/api/python/google/ai/generativelanguage/FunctionCall) 請求。情況如下：

- 使用具有預設 `enable_automatic_function_calling=False` 的 `ChatSession`。
- 使用 `GenerativeModel.generate_content`  (並自行管理聊天記錄)。


以下範例為 Python 中 [函式呼叫單圈次 curl 範例](https://ai.google.dev/docs/function_calling#function-calling-single-turn-curl-sample) 的粗略同義詞。它呼叫具有 (模擬) 電影播放時間資訊，可能來自假想的 API 的函式：


In [ ]:
def find_movies(description: str, location: str = ""):
    """find movie titles currently playing in theaters based on any description, genre, title words, etc.

    Args:
        description: Any kind of description including category or genre, title words, attributes, etc.
        location: The city and state, e.g. San Francisco, CA or a zip code e.g. 95616
    """
    return ["Barbie", "Oppenheimer"]


def find_theaters(location: str, movie: str = ""):
    """Find theaters based on location and optionally movie title which are is currently playing in theaters.

    Args:
        location: The city and state, e.g. San Francisco, CA or a zip code e.g. 95616
        movie: Any movie title
    """
    return ["Googleplex 16", "Android Theatre"]


def get_showtimes(location: str, movie: str, theater: str, date: str):
    """
    Find the start times for movies playing in a specific theater.

    Args:
      location: The city and state, e.g. San Francisco, CA or a zip code e.g. 95616
      movie: Any movie title
      thearer: Name of the theater
      date: Date for requested showtime
    """
    return ["10:00", "11:00"]

使用字典，方便之後按名稱查找功能。你也可以將功能陣列傳遞給 `GenerativeModel` 的 `tools` 參數。


In [ ]:
functions = {
    "find_movies": find_movies,
    "find_theaters": find_theaters,
    "get_showtimes": get_showtimes,
}

model = genai.GenerativeModel(model_name="gemini-1.0-pro", tools=functions.values())

在使用 `generate_content()` 提問後，模型要求 `function_call`：


In [ ]:
response = model.generate_content(
    "Which theaters in Mountain View show the Barbie movie?"
)
response.candidates[0].content.parts

[function_call {
  name: "find_theaters"
  args {
    fields {
      key: "location"
      value {
        string_value: "Mountain View, CA"
      }
    }
    fields {
      key: "movie"
      value {
        string_value: "Barbie"
      }
    }
  }
}
]

由於這不使用帶有自動功能呼叫的 `ChatSession`，你必須自己呼叫該功能。

一種很簡單的方法是使用 `if` 敘述：

```python
if function_call.name == 'find_theaters':
  find_theaters(**function_call.args)
elif ...
```

但是，由於你已經建立 `functions` 字典，因此可以簡化為：


In [ ]:
def call_function(function_call, functions):
    function_name = function_call.name
    function_args = function_call.args
    return functions[function_name](**function_args)


part = response.candidates[0].content.parts[0]

# Check if it's a function call; in real use you'd need to also handle text
# responses as you won't know what the model will respond with.
if part.function_call:
    result = call_function(part.function_call, functions)

print(result)

['Googleplex 16', 'Android Theatre']


最後，將回應與訊息記錄傳遞到下一個 `generate_content()` 呼叫，以從模型取得最終文字回應。


In [ ]:
import google.ai.generativelanguage as glm
from google.protobuf.struct_pb2 import Struct

# Put the result in a protobuf Struct
s = Struct()
s.update({"result": result})

# Update this after https://github.com/google/generative-ai-python/issues/243
function_response = glm.Part(
    function_response=glm.FunctionResponse(name="find_theaters", response=s)
)

# Build the message history
messages = [
    # fmt: off
    {"role": "user",
     "parts": ["Which theaters in Mountain View show the Barbie movie?."]},
    {"role": "model",
     "parts": response.candidates[0].content.parts},
    {"role": "user",
     "parts": [function_response]},
    # fmt: on
]

# Generate the next response
response = model.generate_content(messages)
print(response.text)

Theaters showing Barbie in Mountain View, CA are: Googleplex 16, Android Theatre


## 平行函式呼叫

Gemini API 可以一次呼叫多個函式。這針對有可以獨立發生以完成任務的多個函式呼叫的情境。

先設定工具。不同於上述的電影範例，這些函式不需要輸入才能相互呼叫，所以它們會是很好的平行呼叫候選者。


In [ ]:
def power_disco_ball(power: bool) -> bool:
    """Powers the spinning disco ball."""
    print(f"Disco ball is {'spinning!' if power else 'stopped.'}")
    return True


def start_music(energetic: bool, loud: bool, bpm: int) -> str:
    """Play some music matching the specified parameters.

    Args:
      energetic: Whether the music is energetic or not.
      loud: Whether the music is loud or not.
      bpm: The beats per minute of the music.

    Returns: The name of the song being played.
    """
    print(f"Starting music! {energetic=} {loud=}, {bpm=}")
    return "Never gonna give you up."


def dim_lights(brightness: float) -> bool:
    """Dim the lights.

    Args:
      brightness: The brightness of the lights, 0.0 is off, 1.0 is full.
    """
    print(f"Lights are now set to {brightness:.0%}")
    return True

現在呼叫具有可以運用所有指定工具的指示的模型。


In [ ]:
# Set the model up with tools.
house_fns = [power_disco_ball, start_music, dim_lights]
# Try this out with Pro and Flash...
model = genai.GenerativeModel(model_name="gemini-1.5-pro-latest", tools=house_fns)

# Call the API.
chat = model.start_chat()
response = chat.send_message("Turn this place into a party!")

# Print out each of the function calls requested from this single call.
for part in response.parts:
    if fn := part.function_call:
        args = ", ".join(f"{key}={val}" for key, val in fn.args.items())
        print(f"{fn.name}({args})")

power_disco_ball(power=True)
start_music(energetic=True, loud=True, bpm=120.0)
dim_lights(brightness=0.3)


列印的各位結果皆反映了模型所請求的單一功能呼叫。若要傳回結果，請依據請求的順序包含回應。


In [ ]:
import google.ai.generativelanguage as glm

# Simulate the responses from the specified tools.
responses = {
    "power_disco_ball": True,
    "start_music": "Never gonna give you up.",
    "dim_lights": True,
}

# Build the response parts.
response_parts = [
    glm.Part(function_response=glm.FunctionResponse(name=fn, response={"result": val}))
    for fn, val in responses.items()
]

response = chat.send_message(response_parts)
print(response.text)

Let's get this party started! I've turned on the disco ball, started playing some upbeat music, and dimmed the lights. 🎶✨  Get ready to dance! 🕺💃 




## 後續步驟


實用的 API 參考：

- [genai.GenerativeModel](https://ai.google.dev/api/python/google/generativeai/GenerativeModel) 類別
  - 其 [GenerativeModel.generate_content](https://ai.google.dev/api/python/google/generativeai/GenerativeModel#generate_content) 方法建立 [glm.GenerateContentRequest](https://ai.google.dev/api/python/google/ai/generativelanguage/GenerateContentRequest) 背後的場景。
    - 請求的`.tools` 欄位包含 1 個 [glm.Tool](https://ai.google.dev/api/python/google/ai/generativelanguage/Tool) 物件清單。
    - 該工具的 `function_declarations` 屬性包含 [FunctionDeclarations](https://ai.google.dev/api/python/google/ai/generativelanguage/FunctionDeclaration) 物件清單。
- [回應](https://ai.google.dev/api/python/google/ai/generativelanguage/GenerateContentResponse) 可能包含 [glm.FunctionCall](https://ai.google.dev/api/python/google/ai/generativelanguage/FunctionCall)，在 `response.candidates[0].contents.parts[0]` 中。
- 如果設定 `enable_automatic_function_calling` [genai.ChatSession](https://ai.google.dev/api/python/google/generativeai/ChatSession) 執行呼叫，並送回 [glm.FunctionResponse](https://ai.google.dev/api/python/google/ai/generativelanguage/FunctionResponse)。
- 在回應一個 [FunctionCall](https://ai.google.dev/api/python/google/ai/generativelanguage/FunctionCall) 時，模型總是預期一個 [FunctionResponse](https://ai.google.dev/api/python/google/ai/generativelanguage/FunctionResponse)。
- 如果你使用 [chat.send_message](https://ai.google.dev/api/python/google/generativeai/ChatSession#send_message) 或 [model.generate_content](https://ai.google.dev/api/python/google/generativeai/GenerativeModel#generate_content) 手動回覆記得 API 是無狀態的，你必須發送整個對話記錄 (包含 [內容](https://ai.google.dev/api/python/google/ai/generativelanguage/Content) 物件清單)，而不僅僅是最後一個包含 `FunctionResponse` 的內容。
